In [ ]:
import numpy as np
import h5py
import matplotlob.pyplot as plt
from testCases import *
from dnn_utils import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload # Reload all modules (except those excluded by %aimport) every time before executing the Python code typed.
%autoreload 2

np.random.seed(1)

In [2]:
def initialize_parameters(n_x, n_h, n_y):
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros(shape = (n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros(shape = (n_y, 1))
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters = {
        "W1" : W1,
        "b1" : b1,
        "W2" : W2,
        "b2" : b2
    }
    
    return parameters

In [6]:
def initialize_parameters_deep(layer_dims):
    parameters = {}
    L = len(layer_dims)
    
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

    return parameters

In [9]:
parameters = initialize_parameters_deep([5, 4, 3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + )

W1 = [[-0.00282966  0.0047574  -0.00439517  0.00509468  0.00824658]
 [ 0.00095518 -0.00826851 -0.01098793 -0.01875432 -0.00128511]
 [ 0.01252344 -0.00070233  0.0052842  -0.00430997  0.01200147]
 [ 0.00146411 -0.01317687 -0.00148055  0.01682451  0.00957446]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.00024299 -0.00886032 -0.00435541  0.00695593]
 [ 0.00915528 -0.00162421  0.0067568   0.00563687]
 [ 0.00171798 -0.00609888 -0.01515407 -0.0023212 ]]
b2 = [[0.]
 [0.]
 [0.]]


In [20]:
def linear_forward(A, W, b):
    Z = np.dot(W, A) + b
    cache = (A, W, b)
    return Z, cache

In [14]:
def linear_activation_forward(A_prev, W, b, activation):
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    assert(A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

In [15]:
def sigmoid(Z):  
    A = 1/(1+np.exp(-Z))
    cache = Z
    
    return A, cache

def relu(Z):   
    A = np.maximum(0,Z)
    
    assert(A.shape == Z.shape)
    
    cache = Z 
    return A, cache

In [16]:
def L_model_forward(X, parameters): 
    caches = []
    A = X
    L = len(parameters) // 2 # This gives the number of layers in the NN
    
    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters["b" + str(l)], activation = 'relu')
        caches.append(cache)
    
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str[L]], activation = 'sigmoid')
    caches.append(cache)
    
    assert(AL.shape == (1, X.shape[1]))
    return AL, caches

In [21]:
def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = (-1/m) * np.sum(np.multiply(Y, np.log(AL)) + np.multiply(1 - Y), np.log())
    cost = np.squeeze(cost) # To make sure your cost's shape is what we expect, this turns [[17]] to 17
    assert(cost.shape == ())
    return cost

In [23]:
# BACKWARD PROPAGATION MODULE

def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
        
    dW = np.dot(dZ, A_prev.T) / m
    db = np.squeeze(np.sum(dZ, axis=1, keepdims=True))/m
    dA_prev = np.dot(W.T, dZ)

    assert(dA_prev.shape == A_prev.shape)
    assert(dW.shape == W.shape)
    assert(isinstance(db, float))

    return dA_prev, dW, db
